# Import

In [ ]:
import numpy as np
import pandas as pd
from load_data import loadData
from util import IoU, Dice, masked, remove_small_regions
from keras.models import load_model, model_from_json
from keras.preprocessing.image import ImageDataGenerator
from skimage import morphology, io, color, exposure, img_as_float, transform
from matplotlib import pyplot as plt
import json
import os
import glob

# Configuration

In [ ]:
im_shape = (256, 256,3) # Shape of the image (patch)
mask_shape = (256, 256) # Shape of the msage
model_name = 'model.099.hdf5' # Name of the mode file
path = '/root/workspace/data/mylungrgbtest/' # path of files 
mask_format="_mask" # suffix of the masks without the extension
img_format=".png" # extension of the images

# Code Exection

In [ ]:
# Get the files
# TODO :  Improve maybe ?
files = glob.glob(os.path.join(path,"*3*[!"+mask_format+"]"+img_format))

In [ ]:
# Load test data
X, y = loadData(files, img_format, mask_format)
n_test = X.shape[0]

In [ ]:
# Load model
inp_shape = X[0].shape
print(inp_shape)
# TODO : Check if file exists
UNet = load_model(model_name)

In [ ]:
# Initialization
test_gen = ImageDataGenerator(rescale=1.)
ious = np.zeros(n_test)
dices = np.zeros(n_test)
gts, prs = [], []
i = 0
plt.figure(figsize=(10, 10))

In [ ]:
# Prediction
for xx, yy in test_gen.flow(X, y, batch_size=1):
    img = exposure.rescale_intensity(np.squeeze(xx), out_range=(0,1))
    pred = UNet.predict(xx)[..., 0].reshape(inp_shape[:2])
    mask = yy[..., 0].reshape(inp_shape[:2])
    gt = mask > 0.5
    pr = pred > 0.5
    pr = remove_small_regions(pr, 0.02 * np.prod(im_shape))
    #io.imsave('{}'.format(df.iloc[i].path), masked(img, gt, pr, 1))
    gts.append(gt)
    prs.append(pr)
    ious[i] = IoU(gt, pr)
    dices[i] = Dice(gt, pr)
    print(i, ious[i], dices[i])
    if i < 4:
        plt.subplot(4, 4, 4*i+1)
        plt.title('Processed ' + str(i))
        plt.axis('off')
        plt.imshow(img, cmap='gray')
        plt.subplot(4, 4, 4 * i + 2)
        plt.title('IoU = {:.4f}'.format(ious[i]))
        plt.axis('off')
        plt.imshow(masked(img, gt, pr, 1))
        plt.subplot(4, 4, 4*i+3)
        plt.title('Prediction')
        plt.axis('off')
        plt.imshow(pred, cmap='jet')
        plt.subplot(4, 4, 4*i+4)
        plt.title('Difference')
        plt.axis('off')
        plt.imshow(np.dstack((pr.astype(np.int8), gt.astype(np.int8), pr.astype(np.int8))))
    i += 1
    if i == n_test:
        break
print('Mean IoU:', ious.mean())
print('Mean Dice:', dices.mean())
plt.tight_layout()
plt.savefig('results.png')
plt.show()